# Referenzdatensatz erstellen

### 1. Lösung zweier Formatierungsprobleme

In [7]:
# 1. Besonderheit der HASOC 2020 Daten: Zeilenumbrüche innerhalb der Tweets
# --> die Dateien so aufbereiten, dass nur ein Tweet pro Zeile steht

def rem_white(filename):
    with open(filename, mode="r", encoding="utf-16") as f:
        content = f.readlines()
        content = content[1:] # Erklärungszeile ignorieren
        newcont = []
        for i in range(len(content)):
            # Fall 1: Zeile ist komplett
            if len(content[i].split("\t")) == 5:
                newcont.append(content[i])
            # Fall 2: Zeile ist nicht komplett
            else:
                # letzter Teil einer Zeile erreicht
                if "hasoc_2020_de_" in content[i]:
                    comp_line += content[i]
                    comp_line = comp_line.replace("\n"," ")
                    comp_line += "\n"
                    newcont.append(comp_line)
                # erster Teil einer Zeile 
                elif content[i].startswith("11") :
                    comp_line = content[i]
                # mittlerer Teil einer Zeile, manchmal nur \n
                else: comp_line += content[i]
    nwfilename = filename[:len(filename)-4] + "_formatted" + ".txt"
    with open(nwfilename, mode="w", encoding="utf-8") as outfile:
        for line in newcont:
            outfile.write(line)
    return True

# Bereits formatiert, nicht nochmals durchführen
#rem_white("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_train_new.txt")
#rem_white("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_test_new.txt")

In [24]:
# 2. Besonderheit des GermEval2019-Datensatzes:
# Erstes Zeichen der Tweets in den Testdaten fehlt zum Teil
# Abgeschnitten von GermEval2019 beim Labeln der Daten (von "Testdata_Subtask12" zu "GoldLabelSubtask12")
# z.B. Zeile 341: "enschen, die etwas auf eBay-Kleinanzeigen verticken, ... OTHER	OTHER"
# Aufbereiten: Testdaten mit den Originaltweets aus der Datei "germeval2019_Testdata_Subtask12.txt" speichern
# (Bereits ausgeführt, nicht nochmals durchführen)

with open("..\Korpora\GermEval-2019-Data\germeval2019GoldLabelsSubtask1_2.txt", mode="r", encoding="utf-8") as in_test:
    cont_test = in_test.readlines()
    sep_cont = [line.strip().split("\t") for line in cont_test]

# Tweets ohne abgeschnitte Anfänge einlesen
with open("..\Korpora\GermEval-2019-Data\germeval2019_Testdata_Subtask12.txt", mode="r", encoding="utf-8") as in_tweets:
    tweets = in_tweets.readlines()
    tweets = [tweet.strip() for tweet in tweets]

# Tweets mit den Labels zusammenführen und neu speichern
tweets_replaced = [(tweets[i],sep_cont[i][1],sep_cont[i][2]) for i in range(len(sep_cont))]
with open("..\Korpora\GermEval-2019-Data\germeval2019GoldLabelsSubtask1_2_ersetzt.txt", mode="w", encoding="utf-8") as out_test:
    for line in tweets_replaced:
        out_test.write("\t".join(line)+"\n")

### 2. Angleichen der Formatierung und Annotation

In [1]:
import re

def convert_to_refcorp(filename, corp_id, mod):
    """
    GermEval-Daten und HASOC-Daten in ein einheitliches Format übertragen.

    Input: Datei mit Tabstopp-getrennten Werten, Korpus-ID, train/test-Information
    Output: Liste von Tupeln im Format (Referenzkorpus-ID, Tweet, Label1, Label2)
            - ReferenzkorpusID; setzt sich zusammen aus der Korpus-ID,
                                md_id = "11", falls es um Trainingsdaten (mod=train), "22", falls es um Testdaten (mod=test) geht
                                und der Zeilennummer in der Ursprungsdatei;
                                also z.B.: "01220034" - für einen Tweet der Zeile 34, aus den Testdaten des GermEval2018-Datensatzes
            - Tweet; formatierter Tweet
            - Label1       
            - Label2
    """
    newcorp = []
    with open(filename, mode="r", encoding="utf-8") as f:
        text = f.readlines()
        
        # erste Zeile ignorieren bei HASOC2019 ("03")
        if corp_id == "03": text = text[1:]

        # Bestimmen, welche Formatierungsfunktion genutzt wird
        if corp_id == "01" or corp_id == "02": form_func = format_germeval
        elif corp_id == "03" or corp_id == "04": form_func = format_hasoc

        url_pattern = re.compile('https:\/\/.*?(?: |$)')

        for num, entry in enumerate(text):
            entry = entry.strip()
            tag1, tag2 = "NOT", "NOT"

            tweet, tag1, tag2 = form_func(entry, tag1, tag2)

            # URLs mit generischer Twitter-URL ersetzen
            tweet = url_pattern.sub("https://t.co ", tweet)
            tweet = tweet.strip()

            # Tweet von HTML-Resten entfernen und Emoji-Codierung mit Emojis ersetzen
            tweet = clean_tweet(tweet)

            # gedoppelte und überflüssige Anführungszeichen entfernen
            tweet = tweet.replace('""',"'")
            tweet = tweet.strip('"')

            # ID erstellen
            if mod == "train": md_id = "11"
            elif mod =="test": md_id = "22"
            id_num = f'{num+1:04d}'
            tweet_id = str(corp_id) + str(md_id) + str(id_num)
            
            # der neuen Sammlung hinzufügen
            newcorp.append((tweet_id, tweet, tag1, tag2))
    return newcorp

def format_germeval(entry, tag1, tag2):
    """GermEval-Annotation auf die neue Annotation abbilden & Token |LBR| ersetzen"""
    tweet, label1, label2 = entry.split("\t")
    if label1 == "OFFENSE": tag1 = "NEG"
    if label2 == "INSULT": tag2 = "INSOFF"
    elif label2 == "PROFANITY": tag2 = "PRFN"
    elif label2 == "ABUSE": tag2 = "HATE"
    tweet = tweet.replace("|LBR|", " ")
    return tweet, tag1, tag2


def format_hasoc(entry, tag1, tag2):
    """HASOC-Annotation auf die neue Annotation abbilden"""
    sep = entry.split("\t")
    tweet, l1, l2 = sep[1], sep[2], sep[3]
    if l1 == "HOF": tag1 = "NEG"
    if l2 not in ["HATE", "OFFN", "PRFN"] and l1!="NOT": print(l2)
    if l2 == "HATE": tag2 = "HATE"
    elif l2 == "OFFN": tag2 = "INSOFF"
    elif l2 == "PRFN": tag2 = "PRFN"
    return tweet, tag1, tag2


def clean_tweet(tweet):
    """Emojis finden und ersetzen und HTML-Reste entfernen"""
    cleaned = tweet
    # Emojis, die als Text, z.B. "<U+0001F60A>", gespeichert sind: als utf-8 formatieren
    # s. https://stackoverflow.com/questions/67507017/replace-unicode-code-point-with-actual-character-using-regex
    cleaned = re.sub(r'<U\+([A-F0-9]+)>', lambda x: chr(int(x.group(1), 16)), cleaned)
    cleaned = re.sub(r"&lt;" , "<", cleaned)	 
    cleaned = re.sub(r"&gt;" , ">", cleaned)    
    cleaned = re.sub(r"&amp;" , "&", cleaned)
    cleaned = re.sub(r'\"', '"', cleaned)
    cleaned = re.sub(r'\""', '"', cleaned)
    return cleaned

In [ ]:
# GermEval2018
germeval2018train_converted = convert_to_refcorp("..\Korpora\GermEval-2018-Data-master\germeval2018.training.txt", "01", "train")
germeval2018test_converted = convert_to_refcorp("..\Korpora\GermEval-2018-Data-master\germeval2018.test.txt", "01", "test")

# GermEval2019
germeval2019train_converted = convert_to_refcorp("..\Korpora\GermEval-2019-Data\germeval2019.training_subtask1_2_korrigiert.txt", "02", "train")
germeval2019test_converted = convert_to_refcorp("..\Korpora\GermEval-2019-Data\germeval2019GoldLabelsSubtask1_2_ersetzt.txt", "02", "test")

# HASOC 2019
hasoc2019train_converted = convert_to_refcorp("..\Korpora\german_dataset_hasoc2019\german_dataset\german_dataset.tsv", "03", "train")
hasoc2019test_converted = convert_to_refcorp("..\Korpora\german_dataset_hasoc2019\german_dataset\hasoc_de_test_gold.tsv", "03", "test")

# HASOC 2020
hasoc2020train_converted = convert_to_refcorp("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_train_new_formatted.txt", "04", "train")
hasoc2020test_converted = convert_to_refcorp("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_test_new_formatted.txt", "04", "test")

### 3. Referenzdatensatz zusammenstellen

In [2]:
# Referenzdatensatz zusammenstellen
# (Bereits ausgeführt, nicht nochmals ausführen)

refcorp_train = germeval2018train_converted + germeval2019train_converted + hasoc2019train_converted + hasoc2020train_converted
refcorp_test = germeval2018test_converted + germeval2019test_converted + hasoc2019test_converted + hasoc2020test_converted

import random

random.shuffle(refcorp_train)
random.shuffle(refcorp_test)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train.txt", mode="w", encoding="utf-8") as reftrainout:
    reftrainout.write("corpus_id\ttweet\tbinarylabel\tfinelabel\n")
    for reftweet in refcorp_train:
        reftrainout.write("\t".join(reftweet)+"\n")
    
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test.txt", mode="w", encoding="utf-8") as reftestout:
    reftestout.write("corpus_id\ttweet\tbinarylabel\tfinelabel\n")
    for reftweet in refcorp_test:
        reftestout.write("\t".join(reftweet)+"\n")

### 4. Duplikate entfernen und @user-Erwähnungen anonymisieren

In [8]:
import re

def anonym_atuser(tweet):
    """Ersetzen von @user-Erwähnungen
    Von Inputs der Form '@rspctfl@houelle_beck @ergroovt'
    zu Outputs der Form '@user@user @user'
    """
    tweet_anonym = re.sub('@[^@ ]+?@', '@user@', tweet)
    tweet_anonym = re.sub('@[^@ ]+? ', '@user ', tweet_anonym)
    tweet_anonym = re.sub('@[^@ ]+?$', '@user', tweet_anonym)
    return tweet_anonym

In [77]:
# Berechnung des Jaccard-Index für zwei Multisets und einige Hilfsfunktionen

def jaccard_multisets(bag1, bag2):
    """ Den Jaccard-Index, ein Ähnlichkeitsmaß, für zwei Multisets (von zwei Strings) berechnen.
    Input: zwei Multisets im Format
        {   'e': 3, 'a': 4, ..., 'x': 1,
            'tok': {'e', 'a', ..., 'x'},
            'len': 23 
        }
    Output: Jaccard-Index = Länge der Schnittmenge / Länge der Vereinigung;
            max. 0.5 (sehr ähnlich), min 0.0 (gar nicht ähnlich)
    """
    # Schnittmenge bauen, die Gesamtlänge speichern
    schnitt_len = 0
    schnitt = bag1["tok"] & bag2["tok"]
    for gram in schnitt:
        schnitt_len += min(bag1[gram],bag2[gram])
    # Länge der Vereinigung ermitteln
    vereinigung_len = bag1["len"] + bag2["len"]
    # Jaccard-Index berechnen
    jaccard_index = schnitt_len / vereinigung_len
    return jaccard_index
    

def make_bag(string1):
    """ String als Multiset speichern. Hilfsfunktion zur Beschleunigung der Berechnung des Jaccard-Index für Multisets.

    Input: String
    Output: Dictionary im Format
        {   'e': 3, 'a': 4, ..., 'x': 1,    # jedes Unigramm als Key, die Frequenz als Value
            'tok': {'e', 'a', ..., 'x'},    # das Set aller Unigramme
            'len': 23                       # die Länge des Strings
        }
    """
    stringbag = dict()
    stringset = set(string1)
    for ch1 in stringset: stringbag[ch1] = string1.count(ch1)
    stringbag["len"] = len(string1)
    stringbag["tok"] = stringset
    return stringbag


def calculate_bags(corpus):
    """ Multisets (bags) für alle Strings in einem Korpus berechen.
    Input:  Korpus
    Output: Liste von Multisets (Index in der Liste == Index in der Korpusliste)
    """
    bags = [make_bag(tweet[1]) for tweet in corpus]
    return bags


def jaccard(set1, set2):
    """Jaccard-Index von zwei Sets (von zwei Strings) berechnen.
       Max.: 1 (sehr ähnlich), Min.: 0 (sehr verschieden)
    """
    jaccard_index = len(set1 & set2) / len(set1 | set2)
    return jaccard_index


def calculate_sets(corpus):
    """Set für alle Tweets in einem Korpus berechnen; als Liste der Sets speichern.
    """
    sets = [set(tweet[1]) for tweet in corpus]
    return sets



def collect_duplicates(corpus, sets, simfunc, cutoff):
    """ Duplikate sammeln
    Input:  corpus  - Korpus,
            sets    - Liste von Multisets für alle Tweets im Korpus,
            simfunc - Vergleichsfunktion,
            cutoff  - Ähnlichkeitsgrenzwert
    Output: Liste potentieller Duplikate (als Tupel der Korpuseinträge)
    """
    duplicates = []
    for i in range(len(corpus)):
        if i % 1000 == 0: print("i",i) # Zur Zeitmessung
        for j in range(len(corpus)):
            if i >= j: continue # Kein Vergleich mit sich selbst, und jedes Paar nur in eine Richtung
            else: # Ähnlichkeitswert ermitteln
                jacc = simfunc(sets[i],sets[j])
                if jacc >= cutoff:
                    duplicates.append((corpus[i],corpus[j]))
    return duplicates


def sim_clusters(duplicates):
    """ Aus einer Liste von Duplikaten Duplikat-Cluster bestimmen.
    Input:  Liste von Duplikatpaaren im Format (Text1, Text2);
            mit Text 1 und 2 im Format (Korpus_ID, ...)
    Output: Liste der Clustersets, Anzahl der vorkommenden Tweets
    """
    # Falls ID1 ähnlich ID2: zusammen in ein Set, und jede weitere ID, die einer der beiden IDs ähnlich ist dazu
    clusters = []
    ID_cluster_ref = dict()
    for (tweet1, tweet2) in duplicates:
        #tweet1, tweet2 = duplicate_tup
        ID1, ID2 = tweet1[0], tweet2[0]
        # Fall 1: Weder ID1 noch ID2 vorhanden: neues Cluster
        if (ID1 not in ID_cluster_ref) and (ID2 not in ID_cluster_ref):
            cluster_num = len(clusters)
            ID_cluster_ref[ID1], ID_cluster_ref[ID2] = cluster_num, cluster_num
            clusters.append({tweet1, tweet2})
        # Fall 2: Eine der beiden IDs vorhanden: beide IDs ins vorhandene Cluster integrieren
        elif ID1 in ID_cluster_ref:
            cluster_num = ID_cluster_ref[ID1]
            ID_cluster_ref[ID2] = cluster_num
            clusters[cluster_num].add(tweet2)
        elif ID2 in ID_cluster_ref:
            cluster_num = ID_cluster_ref[ID2]
            ID_cluster_ref[ID1] = cluster_num
            clusters[cluster_num].add(tweet1)
        # Fall 3: Beide bereits vorhanden: Cluster-IDs überprüfen
        else:
            assert ID_cluster_ref[ID1] == ID_cluster_ref[ID2]

    return clusters, len(ID_cluster_ref), ID_cluster_ref.keys()

In [78]:
# Veranschaulichung des Jaccard-Indexes für Mengen und für Multimengen

tweet1 = ""
tweet2 = "@user einigen wir uns doch darauf, dass linker antikommunismus ein nogo ist"
tweet3 = "@user ne mach doch was du willst, kann dem thema eh nicht ausm weg gehen"
tweet4 = "@ThomasOppermann SPD - SCHMAROTZER,PÄDOPHILE UND DENUNZIANTEN   oder   SCHEINHEILGSTE PARTEI DEUTSCHLANDS !!!"
tweet5 = "SPD - SCHMAROTZER, PÄDOPHILE UND DENUNZIANTEN"
tweet6 = "@user - SCHMAROTZER, PÄDOPHILE UND DENUNZIANTEN"
tweet7 = "Wer hier Naziverhalten sieht, hat ein Rederecht. Auf dem Klo. Scheisshausparolen gehören dort hin. 1683 haben wir die Türken davongejagt. Ja"
tweet8 = "Wer hier Naziverhalten sieht, hat ein Rederecht. Auf dem Klo. Scheisshausparolen gehören dort hin. Schon 1683 wurden die Türken davongejagt."

print(f"Jaccard-Index: \t \t       {jaccard(set(tweet2), set(tweet3))}")
print(f"Jaccard-Index für Multimengen: {jaccard_multisets(make_bag(tweet2), make_bag(tweet3))}")


Jaccard-Index: 	 	       0.9523809523809523
Jaccard-Index für Multimengen: 0.36054421768707484


In [79]:
# Datensatz laden
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train.txt", mode="r", encoding="utf-8") as in_train:
    train = in_train.readlines()

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test.txt", mode="r", encoding="utf-8") as in_test:
    test = in_test.readlines()

daten = train[1:] + test[1:] # ohne Erklärungszeile
daten = [entry.strip().split("\t") for entry in daten]

# @user-Erwähnungen anonymisieren (für eine verbesserte Vergleichbarkeit)
daten_anonym = [(entry[0],anonym_atuser(entry[1]),entry[2],entry[3]) for entry in daten]

# Multisets aller Tweets berechnen
bags = calculate_bags(daten_anonym)

In [80]:
# Jaccard-Index berechnen
dups_multisets = collect_duplicates(daten_anonym, bags, jaccard_multisets, 0.47)

i 0
i 1000
i 2000
i 3000
i 4000
i 5000
i 6000
i 7000
i 8000
i 9000
i 10000
i 11000
i 12000
i 13000
i 14000
i 15000
i 16000
i 17000
i 18000
i 19000
i 20000
i 21000
i 22000
i 23000


In [84]:
# Duplikat-Cluster berechnen
clusters, num_dup, num_ids = sim_clusters(dups_multisets)

# Schwarze Liste zu löschender Korpus-IDs erstellen und speichern
schwarzeListe = []
for gruppe in clusters[2:]:
    grp = list(gruppe)
    for tweet in grp[1:]:
        schwarzeListe.append(tweet[0])

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\schwarze_Liste.txt", mode="w", encoding="utf-8") as schwout:
    for id in schwarzeListe:
        schwout.write(id+"\n")

# Informationen zu den ähnlichen Tweets
print(f"Insg. {num_dup} einander in irgendeiner Konstellation ähnliche Tweets") # 320
print(f"Insg. {len(clusters)} Cluster") # 101 Cluster
print(f"Insg. {len(schwarzeListe)} Tweets, die aus dem Datensatz entfernt werden") # 220


# Ohne die Duplikate in neue Dateien schreiben (nicht mehr anonymisiert)
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_OD.txt", mode="w", encoding="utf-8") as out_train:
    for tweet in train:
        if tweet[0] not in schwarzeListe:
            out_train.write(tweet)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_OD.txt", mode="w", encoding="utf-8") as out_test:
    for tweet in test:
        if tweet[0] not in schwarzeListe:
            out_test.write(tweet)

Insg. 320 einander in irgendeiner Konstellation ähnliche Tweets
Insg. 101 Cluster
Insg. 220 Tweets, die aus dem Datensatz entfernt werden
